In [39]:
from playwright.async_api import async_playwright
import asyncio
from dotenv import load_dotenv

In [40]:
load_dotenv(override=True)

True

### Misc
bbc.com/news
```
await page.wait_for_selector('a[data-testid="internal-link"]', timeout=0)
```

In [85]:
links = []
async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        # await page.goto("https://www.bbc.com/news")
        
        await page.goto("https://www.thehindu.com/news/national/", timeout=0)
        print(await page.title())
        await page.wait_for_selector('h3.title a', timeout=0)
        results = []
        links = await page.query_selector_all('h3.title a')
        print("Links")
        print(links)
        print(len(links))

        for link in links[:10]:  # limit for demo
            href = await link.get_attribute("href")
            text = await link.inner_text()
            if href and text:
                results.append({"title": text.strip(), "url": "https://www.thehindu.com/news/national/" + href})

        await browser.close()
        return results
    
# 👇 In Jupyter, use await directly instead of asyncio.run()
headlines = await run()


India Latest News: Top National Headlines Today & Breaking News - The Hindu
Links
[<JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>, <JSHandle preview=JSHandle@node>]
24


In [86]:
headlines[0].keys()

dict_keys(['title', 'url'])

In [87]:
for h in headlines:
    print(f"- {h['title']}: {h['url']}")

- Over 70% of India’s prisoners have not yet been found guilty: SC judge: https://www.thehindu.com/news/national/https://www.thehindu.com/news/national/over-70-of-indias-prisoners-have-not-yet-been-found-guilty-sc-judge/article70256324.ece
- Digital abuse against women journalists cannot be normalised as part of ‘online discourse’: CJI-designate Justice Surya Kant: https://www.thehindu.com/news/national/https://www.thehindu.com/news/national/digital-abuse-against-women-journalists-cannot-be-normalised-as-part-of-online-discourse-cji-designate-justice-surya-kant/article70257030.ece
- With Trump skipping G20, 'Vishwaguru' to certainly attend in person: Congress: https://www.thehindu.com/news/national/https://www.thehindu.com/news/national/with-trump-skipping-g20-vishwaguru-to-certainly-attend-in-person-congress/article70255590.ece
- ‘PM, CM should be ashamed’: Rahul on kids having mid-day meals on newspapers in Madhya Pradesh: https://www.thehindu.com/news/national/https://www.thehindu.c

In [91]:
async def extract_article_text(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=60000)
        await page.wait_for_load_state("domcontentloaded")
        content = await page.locator("article, div.articlebodycontent").inner_text()
        await browser.close()
        return content

In [92]:
text = await extract_article_text(headlines[0]['url'])

In [93]:
print(text)

Vanita Devi, 36 (name changed) is accused of killing her two children aged three and six in 2017. Married off at the age of 16, she spent most of her life amid domestic violence. Abandoned by family for the crime she says she never committed, she spent over five years in jail with no hope.

She finally got bail in 2022, thanks to a team from the Square Circle Clinic, a Fair Trial Programme (FTP) of the NALSAR University of Law, Hyderabad. Since then, Ms. Devi, who is now diagnosed with mental illness, spends most of her time helping patients at a clinic, which agreed to give her shelter at a time her own kin left her to deal with the world.

 Time to unlock electronic tracking technology for undertrial inmates: SC report

Ms. Devi is one of the over 5,000 cases taken up by the Square Circle Clinic. NALSAR initiated the FTP in 2019 and has worked with undertrials at the Nagpur Central Prison and the Yerawada Central Prison(Pune). Square Circle Clinic-NALSAR was previously known as Proje

In [94]:
from openai import OpenAI
import os
import json

client = OpenAI(
    api_key=os.environ.get("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [95]:
def summarize_text(text:str) -> str:
    summarization_prompt = ''' You are an expert news analyst. You are given with a news article.
                        Carefully summarize that article for the user. Be sure to not miss out on any important
                        piece of information. Give only the summary, do not add anything else.
                        Return the response in this json format - 
                        {
                        "summary" : "..."
                        }
                        '''
    
    response = client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=[
        {"role": "system", "content": summarization_prompt},
        {
            "role": "user",
            "content": text
        }
    ]
)
    
    summary = response.choices[0].message.content
    summary = summary.replace("```json","").replace("```","")

    return json.loads(summary)

In [96]:
summarized_response = summarize_text(text)

In [97]:
print(summarized_response['summary'])

The Square Circle Clinic, a Fair Trial Programme (FTP) of NALSAR University of Law, Hyderabad, has highlighted critical issues faced by undertrial inmates in India through a report covering 5,783 cases from 2019-2024. The report reveals that 41.3% of accused undertrials had no lawyer, 77% had no family contact, 72% had not completed school, and 51% lacked necessary documents for trial. Demographically, 52% were under 30, 58% had at least one disability, 67.6% belonged to disadvantaged castes, and 79.8% worked in the unorganised sector. The clinic's efforts led to 1,834 bail applications, 777 case disposals, and the release of 1,388 clients in 2,542 cases, exemplified by Vanita Devi, who was bailed in 2022 after over five years in jail for a crime she denies, now diagnosed with mental illness and helping at a clinic. Supreme Court Justice Vikram Nath described the report's findings as 'disturbing,' noting that over 70% of India's prison population are undertrials, many unaware of their 

### Bringing all functions together

In [ ]:
client = OpenAI(
    api_key=os.environ.get("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


async def get_headlines(page):
    await page.goto("https://www.thehindu.com/news/national/", timeout=0)
    print(await page.title())
    await page.wait_for_selector('h3.title a', timeout=0)
    results = []
    links = await page.query_selector_all('h3.title a')
    
    for link in links[:5]:
        href = await link.get_attribute("href")
        text = await link.inner_text()
        if href and text:
            results.append({"title": text.strip(), "url": "https://www.thehindu.com/news/national/" + href})

    return results


async def extract_headlines_text(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=0)
        await page.wait_for_load_state("domcontentloaded")
        content = await page.locator("article, div.articlebodycontent").inner_text()
        await browser.close()
        return content
    
    
def summarize_news_article(text:str) -> str:
    summarization_prompt = ''' You are an expert news analyst. You are given with a news article.
                        Carefully summarize that article for the user. Be sure to not miss out on any important
                        piece of information. Give only the summary, do not add anything else.
                        Return the response in this json format - 
                        {
                        "summary" : "..."
                        }
                        '''
    
    response = client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=[
        {"role": "system", "content": summarization_prompt},
        {
            "role": "user",
            "content": text
        }
    ]
)
    
    summary = response.choices[0].message.content
    summary = summary.replace("```json","").replace("```","")
    return json.loads(summary)



async def get_news(): 
    news = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        headlines = await get_headlines(page)
        print(f"Total headlines : {len(headlines)}")
        for i, h_data in enumerate(headlines,1):
            print(f"Processing heading # {i}")
            article = await extract_headlines_text(h_data['url'])
            summarized_article = summarize_news_article(article)
            d = {}
            d['url'] = h_data['url']
            d['title'] = h_data['title']
            d['summary'] = summarized_article
            news.append(d)

        await browser.close()

    return news

# 👇 In Jupyter, use await directly instead of asyncio.run()
news_data = await get_news()

India Latest News: Top National Headlines Today & Breaking News - The Hindu
Total headlines : 5
Processing heading # 1
Processing heading # 2
Processing heading # 3
Processing heading # 4
Processing heading # 5


In [129]:
for n in news_data:
    print(f"TITLE : {n['title']}")
    print(f"URL : {n['url']}")
    print(f"SUMMARY: {n['summary']['summary']}")
    print("*"*50)

TITLE : Over 70% of India’s prisoners have not yet been found guilty: SC judge
URL : https://www.thehindu.com/news/national/https://www.thehindu.com/news/national/over-70-of-indias-prisoners-have-not-yet-been-found-guilty-sc-judge/article70256324.ece
SUMMARY: Vanita Devi, an undertrial accused of killing her children, spent over five years in jail before securing bail in 2022 through the Square Circle Clinic, a Fair Trial Programme (FTP) of NALSAR University of Law, Hyderabad. Her case exemplifies the challenges faced by many undertrials, as she was abandoned by family and later diagnosed with mental illness. The Square Circle Clinic, which has handled over 5,000 cases since 2019, released a report detailing disturbing findings: 41.3% of accused had no lawyer, 77% lost family contact, 72% had not completed school, 51% lacked necessary documents, 52% were under 30, and 58% had a disability. Additionally, 67.6% belonged to disadvantaged castes and 79.8% worked in the unorganised sector. 

#### Using max_retries if there is a timeout error

In [140]:
import asyncio
from playwright.async_api import async_playwright, TimeoutError


async def load_page(page:str, url:str, selector:str, max_retries:int=3,timeout:float=60000, load_state:str="domcontentloaded")-> bool: 
    for attempt in (1, max_retries+1):
        try:
            await page.goto(url, timeout=timeout)
            await page.wait_for_load_state(load_state, timeout=timeout)
            if selector:
                await page.wait_for_selector(selector, timeout=timeout)
            return True
        except TimeoutError:
            print(f"Timeout Error while loading {url} - (Attempt {attempt} of {max_retries})")
            if attempt < max_retries:
                await asyncio.sleep(2*attempt)
            else:
                print(f"Failed to load {url} after {max_retries} attempts")
                return False
        except Exception as e:
            print(f"Error while loading {url} - {e}")
            return False

In [ ]:
async def get_headlines(page, top_k:int=5):
    print(f"[INFO] Fetching headlines from https://www.thehindu.com/news/national/")
    status = await load_page(page, "https://www.thehindu.com/news/national/", "h3.title a")
    if not status:
        return []
    
    headlines = []
    links = await page.query_selector_all("h3.title a")
    for link in links[:top_k]:
        href = await link.get_attribute("href")
        text = await link.inner_text()
        if href and text:
            headlines.append({"title": text.strip(), "url": href.strip()})

    return headlines

In [147]:
async def get_article_for_headline(page, url):
    status = await load_page(page, url, selector="article, div.articlebodycontent")
    if not status:
        return ""
    try:
        content = await page.locator("article, div.articlebodycontent").inner_text()
        return content.strip()
    except Exception as e:
        print(f"[ERROR] Failed to extract main text for {url} - {e}")
        return ""

In [151]:
def get_news_summary(text:str) -> str:
    summarization_prompt = ''' You are an expert news analyst. You are given with a news article.
                        Carefully summarize that article for the user. Be sure to not miss out on any important
                        piece of information. Give only the summary, do not add anything else.
                        Return the response in this json format - 
                        {
                        "summary" : "..."
                        }
                        '''
    
    response = client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=[
        {"role": "system", "content": summarization_prompt},
        {
            "role": "user",
            "content": text
        }
    ]
)
    
    summary = response.choices[0].message.content
    summary = summary.replace("```json","").replace("```","")
    return json.loads(summary)

In [160]:
async def get_news(): 
    news = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        headlines = await get_headlines(page)
        print(f"[INFO] Found {len(headlines)} headlines")

        for i, h_data in enumerate(headlines,1):
            print(f"Processing heading # {i}")
            d = {}
            print(f"{i}. {h['title']} -> {h['url']}")
            d['url'] = h_data['url']
            d['title'] = h_data['title']
            article = await get_article_for_headline(page, h['url'])
            print(f"Article:\n {article[:100]}...Total article length {len(article)}")
            if len(article) > 900:
                summarized_article = get_news_summary(article)
                print(f"Summary:\n {summarized_article['summary'][:100]}...Total summary length {len(summarized_article['summary'])}\n")
                d['summary'] = summarized_article['summary']
            else:
                print(f"Summary:\n {article[:100]}...Total summary length {len(article)}\n")
                d['summary'] = article            
            
            print("*"*80)
            news.append(d)

        await browser.close()

    return news

In [161]:
news_summary = await get_news()

[INFO] Found 5 headlines
Processing heading # 1
1. Anti-terror operations across Jammu, raids underway at dozens of places -> https://www.thehindu.com/news/national/jammu-and-kashmir/jammu-anti-terror-operations-underway-raids-pakistan-doda-ramban-kathua-rajouri-security/article70258776.ece
Article:
 Jammu and Kashmir Police on Sunday (November 9, 2025) extended the ongoing crackdown against terror ...Total article length 2889
Summary:
 Jammu and Kashmir Police, in collaboration with the Army, CRPF, and SOG units, extended a massive cr...Total summary length 1020

********************************************************************************
Processing heading # 2
2. Anti-terror operations across Jammu, raids underway at dozens of places -> https://www.thehindu.com/news/national/jammu-and-kashmir/jammu-anti-terror-operations-underway-raids-pakistan-doda-ramban-kathua-rajouri-security/article70258776.ece
Article:
 Jammu and Kashmir Police on Sunday (November 9, 2025) extended the ongo

In [ ]:
# block to test implemetations
async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()
    headlines = await get_headlines(page)
    print(f"Found {len(headlines)} headlines")
    for i,h in enumerate(headlines, start=1):
        print(f"{i}. {h['title']} -> {h['url']}")
        article = await get_article_for_headline(page, h['url'])
        summary = 
        print(f"{article[:100]}...Total article length {len(article)}")

Found 5 headlines
1. NGT directs Rajasthan government to rehabilitate villagers affected by mining -> https://www.thehindu.com/news/national/rajasthan/ngt-directs-rajasthan-government-to-rehabilitate-villagers-affected-by-mining/article70256792.ece
The National Green Tribunal (NGT) has directed the Rajasthan government to constitute a committee of...Total article length 3888
2. Exercise Trishul to boost integrated readiness across multiple domains: Defence Ministry -> https://www.thehindu.com/news/national/exercise-trishul-to-boost-integrated-readiness-across-multiple-domains-defence-ministry/article70257517.ece
Tri-service Exercise Trishul has commenced with mission-focused validations to strengthen integrated...Total article length 2691
3. Death of an intellectually-disabled rape survivor alerts Supreme Court to lapses in victim compensation -> https://www.thehindu.com/news/national/death-of-an-intellectually-disabled-rape-survivor-alerts-supreme-court-to-lapses-in-victim-compensatio

In [153]:
article = "dfgdfhfgndfbhsrgdbnfhbng"
print(f"Article:\n{article[:100]}...Total article length {len(article)}")


Article:
dfgdfhfgndfbhsrgdbnfhbng...Total article length 24
